<a href="https://colab.research.google.com/github/dangcaptkd/CS114.K21-/blob/master/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lấy data từ kaggle!



In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"quyennguyenthinh","key":"78c6c8efa90a3136ef03e49e71fb5d9b"}'}

In [3]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 72 May  8 09:45 kaggle.json


In [4]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [5]:
!unzip news-headlines-dataset-for-sarcasm-detection

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


# new section

In [0]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer

In [0]:
# Loading data from json file
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)

Xem thử 5 hàng đầu của dataset 

In [11]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


Check the null value

In [0]:
print(data.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


# Using linear support vector classifier, Gaussuan Naive Bayes,  Logistic Regression, Random Forest Classifier

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

**clean data**

Ở cột headline sẽ có 1 vài kí tự cần được loại bỏ, dùng regular expression để loại bỏ các kí tự này

In [0]:
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Gán feature và label 

In [0]:
features = data['headline']
labels = data['is_sarcastic']

Steaming data

In [0]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

Vector hóa các tính năng bằng cách sử dụng TF-IDF Vectorizer (chỗ này để làm gì ta??)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

**split data**

In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

**training and testing data**

model 1: Using linear support vector classifier

In [19]:
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
#print('train:', lsvc.loss(features_train, labels_train))
#core = lsvc.evaluate(features_test, labels_test) 
print('train:', lsvc.score(features_train, labels_train)) 
print('test:', lsvc.score(features_test, labels_test))   

train: 0.9093524612777362
test: 0.8375748502994012


model 2: Using Gaussuan Naive Bayes

In [0]:
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('train:', gnb.score(features_train, labels_train))  
print('test:', gnb.score(features_test, labels_test))    

train: 0.7886335868836952
test: 0.7380239520958084


model 3: Logistic Regression

In [0]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('train:', lr.score(features_train, labels_train))   
print('test:', lr.score(features_test, labels_test))     

train: 0.8816458440074094
test: 0.8308383233532934


model 4: Random Forest Classifier

In [0]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('train:', rfc.score(features_train, labels_train)) 
print('test:', rfc.score(features_test, labels_test))    

train: 0.9882946439128207
test: 0.7971556886227545


# LSTM, GRU, Conv1d model (Sequential)

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.layers import GlobalMaxPool1D, GRU
from keras import optimizers

In [0]:
MAX_SEQ_LENGTH = 40
MAX_WORDS = 20000

In [0]:
tok = Tokenizer(num_words = MAX_WORDS) # keeping 10000 now for first iteration
tok.fit_on_texts(data.headline)
seqs = tok.texts_to_sequences(data.headline)

In [0]:
df = pad_sequences(seqs, MAX_SEQ_LENGTH)
labels = np.asarray(data.is_sarcastic)

In [65]:
print('df.shape:', df.shape)
print('lables.shape:', labels.shape)

df.shape: (26709, 40)
lables.shape: (26709,)


In [0]:
training_samples  = 24000
validation_samples = 2709

In [0]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
df = df[indices]
labels = labels[indices]
features_train = df[:training_samples]
labels_train = labels[:training_samples]
features_test = df[training_samples: training_samples + validation_samples]
labels_test = labels[training_samples: training_samples + validation_samples]

In [68]:
print('features_train:', features_train.shape)
print('labels_train:', labels_train.shape)
print('features_test:', features_test.shape)
print('labels_test:', labels_test.shape)

features_train: (24000, 40)
labels_train: (24000,)
features_test: (2709, 40)
labels_test: (2709,)


In [69]:
EMB_DIM = 6
def fcmodel():
    model = Sequential()
    model.add(Embedding(input_dim=MAX_WORDS, output_dim= EMB_DIM, input_length=MAX_SEQ_LENGTH))    
    
    # Flatten Layer
    model.add(Flatten())
    
    # FC1
    model.add(Dense(64, activation='relu'))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # print model summary
    model.summary()
    
    # When using pretrained embeddings
    #model.layers[0].set_weights([embedding_matrix])
    #model.layers[0].trainable = False
              
    # Compile the model
    model.compile(optimizer = 'rmsprop',
                 loss = 'binary_crossentropy',
                 metrics = ['acc'])
    return model

fcmod = fcmodel()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 6)             120000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 240)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                15424     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 135,489
Trainable params: 135,489
Non-trainable params: 0
_________________________________________________________________


In [0]:
EPOCHS = 9
BATCH_SIZE =512

In [71]:
fchist = fcmod.fit(features_train, labels_train,
                   epochs = EPOCHS,
                   batch_size = BATCH_SIZE,
                   validation_data = (features_test, labels_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24000 samples, validate on 2709 samples
Epoch 1/9
24000/24000 [==============================] - 0s 10us/step - loss: 0.6541 - acc: 0.6070 - val_loss: 0.5957 - val_acc: 0.7047
Epoch 2/9
24000/24000 [==============================] - 0s 7us/step - loss: 0.4943 - acc: 0.8282 - val_loss: 0.4306 - val_acc: 0.8439
Epoch 3/9
24000/24000 [==============================] - 0s 7us/step - loss: 0.3458 - acc: 0.8751 - val_loss: 0.3480 - val_acc: 0.8549
Epoch 4/9
24000/24000 [==============================] - 0s 7us/step - loss: 0.2716 - acc: 0.8959 - val_loss: 0.3187 - val_acc: 0.8590
Epoch 5/9
24000/24000 [==============================] - 0s 5us/step - loss: 0.2293 - acc: 0.9104 - val_loss: 0.3095 - val_acc: 0.8649
Epoch 6/9
24000/24000 [==============================] - 0s 6us/step - loss: 0.1964 - acc: 0.9261 - val_loss: 0.3042 - val_acc: 0.8712
Epoch 7/9
24000/24000 [==============================] - 0s 6us/step - loss: 0.1720 - acc: 0.9361 - val_loss: 0.3082 - val_acc: 0.8749
Epoch

In [56]:
fchist.history['val_acc']

[0.7109634280204773,
 0.8431155681610107,
 0.8615725636482239,
 0.8704319000244141,
 0.8726467490196228,
 0.8722776174545288,
 0.8663713335990906,
 0.8693244457244873,
 0.8641565442085266]

In [0]:
def lstm():
    model = Sequential()
    
    model.add(Embedding(input_dim=MAX_WORDS, output_dim=EMB_DIM, input_length=MAX_SEQ_LENGTH))
    
    model.add(Bidirectional(LSTM(16, return_sequences=True, recurrent_dropout=0.1, dropout=0.1)))
    
    model.add(Bidirectional(LSTM(32, recurrent_dropout=0.1, dropout=0.1)))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer = 'rmsprop',
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])
    
    return model

lsmod = lstm()

In [77]:
lshist = lsmod.fit(features_train, labels_train,
         epochs = EPOCHS,
         batch_size = BATCH_SIZE,
         validation_data = (features_test, labels_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24000 samples, validate on 2709 samples
Epoch 1/9
24000/24000 [==============================] - 16s 659us/step - loss: 0.6736 - acc: 0.5745 - val_loss: 0.5979 - val_acc: 0.5943
Epoch 2/9
24000/24000 [==============================] - 14s 597us/step - loss: 0.4606 - acc: 0.7977 - val_loss: 0.3773 - val_acc: 0.8350
Epoch 3/9
24000/24000 [==============================] - 15s 609us/step - loss: 0.3344 - acc: 0.8607 - val_loss: 0.3553 - val_acc: 0.8391
Epoch 4/9
24000/24000 [==============================] - 14s 602us/step - loss: 0.2634 - acc: 0.8968 - val_loss: 0.3255 - val_acc: 0.8560
Epoch 5/9
24000/24000 [==============================] - 14s 594us/step - loss: 0.2166 - acc: 0.9180 - val_loss: 0.3330 - val_acc: 0.8627
Epoch 6/9
24000/24000 [==============================] - 14s 601us/step - loss: 0.1814 - acc: 0.9323 - val_loss: 0.3264 - val_acc: 0.8616
Epoch 7/9
24000/24000 [==============================] - 15s 611us/step - loss: 0.1578 - acc: 0.9429 - val_loss: 0.3516 - v

In [0]:
def conv1d():
    
    model = Sequential()
    
    model.add(Embedding(input_dim=MAX_WORDS,output_dim=EMB_DIM, input_length=MAX_SEQ_LENGTH))
    
    model.add(Conv1D(filters=32, kernel_size=7, activation='relu'))
    
    model.add(MaxPool1D(pool_size=5))
    
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    
    model.add(Dropout(0.1))
    
    model.add(GlobalMaxPool1D())
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics= ['acc'])
    
    return model

convmod = conv1d()

In [79]:
convhist = convmod.fit(features_train, labels_train,
                      epochs = 20,
                      batch_size = BATCH_SIZE,
                      validation_data = (features_test, labels_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24000 samples, validate on 2709 samples
Epoch 1/20
24000/24000 [==============================] - 5s 209us/step - loss: 0.6797 - acc: 0.5620 - val_loss: 0.6643 - val_acc: 0.5869
Epoch 2/20
24000/24000 [==============================] - 0s 10us/step - loss: 0.6001 - acc: 0.6892 - val_loss: 0.5370 - val_acc: 0.7770
Epoch 3/20
24000/24000 [==============================] - 0s 9us/step - loss: 0.4477 - acc: 0.8126 - val_loss: 0.4323 - val_acc: 0.8014
Epoch 4/20
24000/24000 [==============================] - 0s 9us/step - loss: 0.3556 - acc: 0.8429 - val_loss: 0.3978 - val_acc: 0.8106
Epoch 5/20
24000/24000 [==============================] - 0s 9us/step - loss: 0.3024 - acc: 0.8650 - val_loss: 0.3847 - val_acc: 0.8165
Epoch 6/20
24000/24000 [==============================] - 0s 9us/step - loss: 0.2674 - acc: 0.8794 - val_loss: 0.3860 - val_acc: 0.8199
Epoch 7/20
24000/24000 [==============================] - 0s 10us/step - loss: 0.2418 - acc: 0.8910 - val_loss: 0.3937 - val_acc: 0.

In [0]:
def convgru():
    
    model = Sequential()
    
    model.add(Embedding(input_dim=MAX_WORDS,output_dim=EMB_DIM, input_length=MAX_SEQ_LENGTH))
    
    model.add(Conv1D(filters=32, kernel_size=7, activation='relu'))
    
    model.add(MaxPool1D(pool_size=5))
    
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    
    model.add(GRU(32, dropout=0.1, recurrent_dropout=0.5))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics= ['acc'])
    
    return model

convgrumod = convgru()

In [82]:
convgruhist = convgrumod.fit(features_train, labels_train, 
                           epochs = 9,
                           batch_size=BATCH_SIZE,
                           validation_data = (features_test, labels_test))

Train on 24000 samples, validate on 2709 samples
Epoch 1/9
24000/24000 [==============================] - 1s 40us/step - loss: 0.2176 - acc: 0.9054 - val_loss: 0.4310 - val_acc: 0.8191
Epoch 2/9
24000/24000 [==============================] - 1s 34us/step - loss: 0.1987 - acc: 0.9131 - val_loss: 0.4543 - val_acc: 0.8140
Epoch 3/9
24000/24000 [==============================] - 1s 33us/step - loss: 0.1814 - acc: 0.9227 - val_loss: 0.4875 - val_acc: 0.8066
Epoch 4/9
24000/24000 [==============================] - 1s 32us/step - loss: 0.1666 - acc: 0.9278 - val_loss: 0.4992 - val_acc: 0.8132
Epoch 5/9
24000/24000 [==============================] - 1s 32us/step - loss: 0.1537 - acc: 0.9346 - val_loss: 0.5308 - val_acc: 0.8092
Epoch 6/9
24000/24000 [==============================] - 1s 33us/step - loss: 0.1406 - acc: 0.9400 - val_loss: 0.5896 - val_acc: 0.8058
Epoch 7/9
24000/24000 [==============================] - 1s 31us/step - loss: 0.1309 - acc: 0.9435 - val_loss: 0.6085 - val_acc: 0.7988